# Homework set 2

Please **submit this Jupyter notebook through Canvas** no later than **Mon Nov. 13, 9:00**. **Submit the notebook file with your answers (as .ipynb file) and a pdf printout. The pdf version can be used by the teachers to provide feedback. A pdf version can be made using the save and export option in the Jupyter Lab file menu.**

Homework is in **groups of two**, and you are expected to hand in original work. Work that is copied from another group will not be accepted.

# Exercise 0
Write down the names + student ID of the people in your group.

Loes Bijman 15211312

Sacha Gijsbers 12798525

## Importing packages
Execute the following statement to import the packages `numpy`, `math` and `scipy.sparse`. If additional packages are needed, import them yourself.

In [2]:
import math
import numpy as np
import scipy.sparse as sp
import scipy
from fractions import Fraction
import time

# Sparse matrices

A matrix is called sparse if only a small fraction of the entries is nonzero. For such matrices, special data formats exist. `scipy.sparse` is the scipy package that implements such data formats and provides functionality such as the LU decomposition (in the subpackage `scipy.sparse.linalg`).

As an example, we create the matrix 
$$\begin{bmatrix} 1 & 0 & 2 & 0\\ 
0 & 3 & 0 & 0 \\
0 & 0 & 4 & 5 \\
0 & 0 & 0 & 6 
\end{bmatrix}$$ in the so called compressed sparse row (CSR) format. As you can see, the arrays `row`, `col`, `data` contain the row and column coordinate and the value of each nonzero element respectively.

In [3]:
# a sparse matrix with 6 nonzero entries
row = np.array([0, 0, 1, 2, 2, 3])
col = np.array([0, 2, 1, 2, 3, 3])
data = np.array([1.0, 2, 3, 4, 5, 6])
sparseA = sp.csr_array((data, (row, col)), shape=(4, 4))

# convert to a dense matrix. This allows us to print to screen in regular formatting
denseA = sparseA.toarray()
print(denseA)


[[1. 0. 2. 0.]
 [0. 3. 0. 0.]
 [0. 0. 4. 5.]
 [0. 0. 0. 6.]]


For sparse matrices, a sparse data format is much more efficient in terms of storage than the standard array format. Because of this efficient storage, very large matrices of size $n \times n$ with $n = 10^7$ or more can be stored in RAM for performing computations on regular computers. Often the number of nonzero elements per row is quite small, such as 10's or 100's nonzero elements per row. In a regular, dense format, such matrices would require a supercomputer or could not be stored.

In the second exercise you have to use the package `scipy.sparse`, please look up the functions you need (or ask during class).

# Heath computer exercise 2.1

## (a)
Show that the matrix
$$ A = \begin{bmatrix} 
0.1 & 0.2 & 0.3 \\
0.4 & 0.5 & 0.6 \\
0.7 & 0.8 & 0.9
\end{bmatrix}.$$
is singular. Describe the set of solutions to the system $A x = b$ if
$$ b = \begin{bmatrix} 0.1 \\ 0.3 \\ 0.5 \end{bmatrix}. $$
(N.B. this is a pen-and-paper question.)


--------------------------------------------------
We can see that

$\rm{det}(A) = 0.1\cdot(0.5\cdot 0.9 - 0.6\cdot 0.8) - 0.2(0.4\cdot 0.9 - 0.6\cdot 0.7) + 0.3\cdot(0.4\cdot 0.8 - 0.5\cdot 0.7) = 0$

Therefore matrix $A$ is singular.

Since $A$ is singular, we know there is a nonzero vector $\textbf{z}$ such that $A\textbf{z} = \textbf{0}$. Thus, if $\textbf{x}$ is a solution to $A\textbf{x} = \textbf{b}$, this means $\textbf{x} + \gamma \cdot \textbf{z}$ is also a solution, for any $\gamma \in \mathbb{R}$.

LU decomposition for matrix $A$ gives $L = \begin{bmatrix} 1 & 0 & 0 \\ 4 & 1 & 0 \\ 7 & -2 & 1 \end{bmatrix}$ and $U = \begin{bmatrix} 0.1 & 0.2 & 0.3 \\ 0 & -0.3 & -0.6 \\ 0 & 0 & 0 \end{bmatrix}$. Using these matrices and performing forward and backward substitution, we find $\textbf{z} = \begin{bmatrix} 1 \\ -2 \\ 1 \end{bmatrix}$ and $\textbf{x} = \begin{bmatrix} 1/3 \\ 1/3 \\ 0 \end{bmatrix}$. 

Therefore the general solution is $x = \begin{bmatrix} 1/3 \\ 1/3 \\ 0 \end{bmatrix} + \gamma \begin{bmatrix} 1 \\ -2 \\ 1 \end{bmatrix} = \begin{bmatrix} 1/3 + \gamma \\ 1/3 - 2\gamma \\ \gamma \end{bmatrix}$ for all $\gamma \in \mathbb{R}$ (or in $\mathbb{C}$).

## (b)
If we were to use Gaussian elimination with partial pivoting to solve this system using exact arithmetic, at what point would the process fail?


In the context of LU (Lower-Upper) factorization, which is used for solving linear systems of equations, it is assumed that the original matrix A can be decomposed into two matrices, L and U, such that A = LU. L is a lower triangular matrix, all entries above the main diagonal are zero. U is an upper triangular matrix, all entries below the main diagonal are zero.

 In exact arithmetic, LU factorization, like Gaussian elimination with partial pivoting, is a numerically stable method for solving systems of linear equations. However, if the original matrix A is singular, it can indeed lead to a zero on the diagonal of the upper triangular factor U, hich may result in a failure during the back-substitution step due to division by zero when solving for the variables in the system of equations. The LU factorization would still work but solution process would fail.

## (c)
Because some of the entries of $A$ are not exactly representable in a binary floating point system, the matrix is no longer exactly singular when entered into a computer; thus, solving the system by Gaussian elimination will not necessarily fail. Solve this system on a computer using a library routine for Gaussian elimination. Compare the computed solution with your description of the solution set in part (a). What is the estimated value for $\text{cond}(A)$? How many digits of accuracy in the solution would this lead you to expect?

In [4]:
# sparse matrix A
row = np.array([0, 0, 0, 1, 1, 1, 2, 2, 2])
col = np.array([0, 1, 2, 0, 1, 2, 0, 1, 2])
data = np.array([0.1, 0.2, 0.3, 0.4, 0.5, 0.6, 0.7, 0.8, 0.9])
sparseA = sp.csr_array((data, (row, col)), shape=(3, 3))

# convert to dense matrix
A = sparseA.toarray()
print(f"Matrix A:\n {A}")

b = np.array([0.1, 0.3, 0.5])
print(f"b: {b}")

# Perform Gaussian elimination with partial pivoting
lu, piv = scipy.linalg.lu_factor(A)
x = scipy.linalg.lu_solve((lu, piv), b)

print(f"Solution set x: {x}")

Matrix A:
 [[0.1 0.2 0.3]
 [0.4 0.5 0.6]
 [0.7 0.8 0.9]]
b: [0.1 0.3 0.5]
Solution set x: [ 0.16145833  0.67708333 -0.171875  ]


In [5]:
# Comparison of computed solution with calculation of 2.1(a)

gamma = x[2] # using vectors from 2.1(a)
x_1 = 1/3 + gamma
x_2 = 1/3 - 2*gamma
x_3 = gamma

solution = np.array([x_1,x_2,x_3])

print(f"Solution from 2.1(a) with gamma = {gamma}: {solution}")
print(f"Computed solution set: {x}")


Solution from 2.1(a) with gamma = -0.171875: [ 0.16145833  0.67708333 -0.171875  ]
Computed solution set: [ 0.16145833  0.67708333 -0.171875  ]


We can see that the solution found by the computer fits the general solution as found in 2.1(a) for $\gamma = -0.171875$.

In [23]:
condA = np.linalg.cond(A, p = np.inf)
print("Condition number of A with infinity norm:", condA)
print(f"cond(A)*machine_epsilon = {condA * 2*10**(-16)}")
print(f"log(cond(A)) = {math.log(condA,10)}")

Condition number of A with infinity norm: 8.64691128455135e+16
cond(A)*machine_epsilon = 17.2938225691027
log(cond(A)) = 16.93686100323057


From Heath, we know that $\frac{||\textbf{\^{x}}-\textbf{x}||}{||\textbf{x}||}\lessapprox \rm{cond}(A) \epsilon_{\rm{mach}}$. For double precision we have $\epsilon_{\rm{mach}} \approx 2\cdot 10^{-16}$. In our case, we obtain $\frac{||\textbf{\^{x}}-\textbf{x}||}{||\textbf{x}||}\lessapprox 8.6469 \cdot 10^{16} \cdot 2\cdot 10^{-16} \approx 17.29$. Furthermore, we can see that $\rm{log}_{10}(\rm{cond}(A)) \approx \rm{log}_{10}(8.6469 \cdot 10^{16}) \approx 16.9$ decimal digits of accuracy are lost relative to the accuracy of the input. Therefore we expect no correct digits in the solution unless the input data is accurate to more than 17 digits. Since only 16 digits can be accurately stored, the solution has lost all accuracy.

# Heath computer exercise 2.17

Consider a horizontal cantilevered beam that is clamped at one end but free along the remainder of its length. A discrete model of the forces on the beam yields a system of linear equations $A x = b$, where the $n \times n$ matrix $A$ has the banded form
$$
\begin{bmatrix}
 9 & -4     &  1 &  0 & \ldots & \ldots & 0 \\
-4 &  6     & -4 &  1 & \ddots && \vdots \\
 1 & -4     &  6 & -4 &  1 & \ddots & \vdots \\
 0 & \ddots & \ddots & \ddots & \ddots & \ddots & 0 \\
 \vdots & \ddots & 1 & -4 &  6 & -4 &  1 \\ 
 \vdots && \ddots    &  1 & -4 &  5 & -2 \\
 0 & \ldots & \ldots & 0 & 1 & -2 & 1 
\end{bmatrix}, $$
the $n$-vector $b$ is the known load on the bar (including its own weight), and the $n$-vector $x$ represents the resulting deflection of the bar that is to be determined. We will take the bar to be uniformly loaded, with $b_i = 1/n^4$ for each component of the load vector.


## (a)
Make a python function that creates the matrix $A$ given the size $n$.

In [7]:
import scipy

n = 16

def create_beam_matrix_A(n):
    b = np.ones(n)
    A = scipy.sparse.diags([b, -4 * b, 6 * b, -4 * b, b], [-2, -1, 0, 1, 2], (n, n), format='csc')
    A[0, 0] = 9
    A[n - 2, n - 2] = 5
    A[n - 1, n - 1] = 1
    A[n - 1, n - 2] = -2
    A[n - 2, n - 1] = -2

    A_dense = A.toarray()

    return A_dense

A_dense = create_beam_matrix_A(n)
print(A_dense)


[[ 9. -4.  1.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.]
 [-4.  6. -4.  1.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.]
 [ 1. -4.  6. -4.  1.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.]
 [ 0.  1. -4.  6. -4.  1.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.]
 [ 0.  0.  1. -4.  6. -4.  1.  0.  0.  0.  0.  0.  0.  0.  0.  0.]
 [ 0.  0.  0.  1. -4.  6. -4.  1.  0.  0.  0.  0.  0.  0.  0.  0.]
 [ 0.  0.  0.  0.  1. -4.  6. -4.  1.  0.  0.  0.  0.  0.  0.  0.]
 [ 0.  0.  0.  0.  0.  1. -4.  6. -4.  1.  0.  0.  0.  0.  0.  0.]
 [ 0.  0.  0.  0.  0.  0.  1. -4.  6. -4.  1.  0.  0.  0.  0.  0.]
 [ 0.  0.  0.  0.  0.  0.  0.  1. -4.  6. -4.  1.  0.  0.  0.  0.]
 [ 0.  0.  0.  0.  0.  0.  0.  0.  1. -4.  6. -4.  1.  0.  0.  0.]
 [ 0.  0.  0.  0.  0.  0.  0.  0.  0.  1. -4.  6. -4.  1.  0.  0.]
 [ 0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  1. -4.  6. -4.  1.  0.]
 [ 0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  1. -4.  6. -4.  1.]
 [ 0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  1. -4.  5. 

## (b)

Solve this linear system using both a standard library routine for dense linear systems and a library routine designed for sparse linear systems. Take $n=100$ and $n=1000$. How do the two routines compare in the time required to compute the solution? And in the memory occupied by the LU decomposition? (Hint: as part of this assignment, look for the number of nonzero elements in the matrices $L$ and $U$ of the sparse LU decomposition.)

In [24]:
import time
b = np.full(n, 1/(n^4))

def time_dense_sparse(n,b):
    A_dense = create_beam_matrix_A(n)
    A_sparse = scipy.sparse.csr_matrix(create_beam_matrix_A(n))

    start_dense = time.time()
    lu_dense, piv_dense = scipy.linalg.lu_factor(A_dense)
    x_dense = scipy.linalg.lu_solve((lu_dense, piv_dense), b)
    end_dense = time.time()
    time_dense = end_dense-start_dense

    start_sparse = time.time()
    lu_sparse = scipy.sparse.linalg.splu(A_sparse)
    x_sparse = lu_sparse.solve(b)
    end_sparse = time.time()
    time_sparse = end_sparse-start_sparse
    
    return time_dense, time_sparse, lu_dense, lu_sparse, x_dense, x_sparse


In [9]:
import sys

runs = 100
n_list = [100, 1000]
for n in n_list:
    all_time_dense, all_time_sparse = [],[]
    all_nonzero_dense, all_nonzero_sparse = [], []
    for _ in range(runs):
        time_dense, time_sparse, lu_dense, lu_sparse, _, _ = time_dense_sparse(n,b)
        all_time_dense.append(time_dense)
        all_time_sparse.append(time_sparse)

        num_elements_dense = n*n
        size_dense = sys.getsizeof(lu_dense)

        num_elements_sparse = (lu_sparse.L != 0).sum() + (lu_sparse.U != 0).sum()
        size_sparse = sys.getsizeof(lu_sparse.L != 0) + sys.getsizeof(lu_sparse.U != 0)

    mean_time_dense = np.mean(all_time_dense)
    std_time_dense = np.std(all_time_dense)

    mean_time_sparse = np.mean(all_time_sparse)
    std_time_sparse = np.std(all_time_sparse)

    print(f"n = {n} ({runs} runs): dense average running time = {mean_time_dense} seconds with std {std_time_dense}")
    print(f"n = {n} ({runs} runs): sparse average running time = {mean_time_sparse} seconds with std {std_time_sparse}")

    print(f"For n = {n}, sparse is {mean_time_dense - mean_time_sparse} seconds faster than dense")
    print('---------------------------------------------------')

    print(f"n = {n} ({runs} runs): dense memory occupation: {num_elements_dense} elements and size {size_dense}")
    print(f"n = {n} ({runs} runs): sparse memory occupation: {num_elements_sparse} elements and size {size_sparse}")

    print('---------------------------------------------------')

/Users/loesbijman/anaconda3/envs/my-env/lib/python3.12/site-packages/scipy/sparse/linalg/_dsolve/linsolve.py:412: SparseEfficiencyWarning: splu converted its input to CSC format
  warn('splu converted its input to CSC format', SparseEfficiencyWarning)


n = 100 (100 runs): dense average running time = 0.001464402675628662 seconds with std 0.0030169513626817543
n = 100 (100 runs): sparse average running time = 0.0006986618041992188 seconds with std 0.0006130344464349809
For n = 100, sparse is 0.0007657408714294433 seconds faster than dense
---------------------------------------------------
n = 100 (100 runs): dense memory occupation: 10000 elements and size 80128
n = 100 (100 runs): sparse memory occupation: 691 elements and size 96
---------------------------------------------------
n = 1000 (100 runs): dense average running time = 0.020349955558776854 seconds with std 0.0013924381911442853
n = 1000 (100 runs): sparse average running time = 0.001174635887145996 seconds with std 6.013545741531516e-05
For n = 1000, sparse is 0.019175319671630858 seconds faster than dense
---------------------------------------------------
n = 1000 (100 runs): dense memory occupation: 1000000 elements and size 8000128
n = 1000 (100 runs): sparse memory 

We can see that the method for sparse LU decomposition is faster than the method for dense LU decomposition. When considering size $n$ of the matrix, LU decomposition is slower for both the dense and sparse method. The difference between the two methods is larger when $n$ is larger. We can see that the running time for LU decomposition with a dense matrix increases more than the running time for the sparse method.

When looking at the memory occupied by the LU decompositions, we can see that the dense LU decomposition uses approximately 80000 bytes more then then the ssparse LU decomposition for $n = 100$. For $n = 1000$ the memory occupied by the sparse LU decomposition stays the same, whereas the memory occupied by the dense LU decomposition increases with a factor of 10. Since the dense matrix method stores the zero-values explicitly in all matrices, this method uses much more memory. This is also why the memory occupied by the sparse LU decomposition does not change with a bigger n, as with a bigger n more zero values are added to the matrix, while the non zero values stay the same, as they are not stored in the sparse LU decomposition, the same amount of memory is occupied for both values of $n$.

## (c)
For $n=100$, what is the condition number? What accuracy do you expect based on the condition number?

In [22]:
n = 100
A_dense = create_beam_matrix_A(n)

condA_dense = np.linalg.cond(A_dense, p = np.inf)
print("Condition number of A dense with infinity norm:", condA_dense)
print(f"cond(A)*machine_epsilon = {condA_dense * 2*10**(-16)}")
print(f"log(cond(A)) = {math.log(condA_dense,10)}")

Condition number of A dense with infinity norm: 200666800.074768
cond(A)*machine_epsilon = 4.01333600149536e-08
log(cond(A)) = 8.302475525267644


For $n = 100$ we obtain $\rm{cond}(A) \approx 200666800.0746656$. The relative error in the approximate solution $\textbf{\^{x}}$ is therefore approximately bounded by $\rm{cond}(A) \epsilon_{\rm{mach}} \approx 4.0133 \cdot 10^{-8}$. Therefore we expect to lose about $\rm{log}_{10}(\rm{cond}(A)) \approx \rm{log}_{10}(200666800.0746656) \approx 8.3$ digits of accuracy relative to the accuracy of the input. Thus we expect no correct digits unless the input data and working memory are accurate to more than 8 decimal digits. 

## (d)
How well do the answers of (b) agree with each other (make an appropriate quantitative comparison)?

Should we be worried about the fact that the two answers are different?

In [29]:
_,_,_,_,x_dense, x_sparse = time_dense_sparse(100,b)

# print(x_dense - x_sparse)

def relative_residual(x_hat,A,b):
    r = b-A.dot(x_hat)
    relative_res = np.linalg.norm(r)/(np.linalg.norm(A)*np.linalg.norm(b))
    return relative_res

print(f"relative residual for dense method: {relative_residual(x_dense,A_dense,b)}")
print(f"relative residual for sparse method:{relative_residual(x_sparse,A_dense,b)}")

relative residual for dense method: 4.3991116791007416e-11
relative residual for sparse method:4.7239361960273103e-11


/Users/loesbijman/anaconda3/envs/my-env/lib/python3.12/site-packages/scipy/sparse/linalg/_dsolve/linsolve.py:412: SparseEfficiencyWarning: splu converted its input to CSC format
  warn('splu converted its input to CSC format', SparseEfficiencyWarning)


your answer (text) here